<a href="https://colab.research.google.com/github/smruthyunjaya/levisTakeHomeAssignment/blob/dev/levisAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Levis Take Home Assignment

 # Git setup

In [1]:
!git clone -b dev https://github.com/smruthyunjaya/levisTakeHomeAssignment.git
!cd levisTakeHomeAssignment/



Cloning into 'levisTakeHomeAssignment'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [9]:
import sys
import os
sys.path.append(os.path.abspath('/content/levisTakeHomeAssignment/'))

In [18]:
from data_utils.DataLoader import DataLoader

In [25]:
dl = DataLoader('https://docs.google.com/spreadsheets/d/e/2PACX-1vSag0RkuAVTVHmtxQHPAPc-6EGGKzWF9bkVq0xiH2AnyUcy_7VQwZGm-TFWZSVvxTlOFL3N1rYXTx5S/pub?gid=646522252&single=true&output=tsv')
dl.print_summary()
df = dl.df

(10127, 21)
          CLIENTNUM  Customer_Age  Dependent_count  Months_on_book  \
count  1.012700e+04  10127.000000     10127.000000    10127.000000   
mean   7.391776e+08     46.325960         2.346203       35.928409   
std    3.690378e+07      8.016814         1.298908        7.986416   
min    7.080821e+08     26.000000         0.000000       13.000000   
25%    7.130368e+08     41.000000         1.000000       31.000000   
50%    7.179264e+08     46.000000         2.000000       36.000000   
75%    7.731435e+08     52.000000         3.000000       40.000000   
max    8.283431e+08     73.000000         5.000000       56.000000   

       Total_Relationship_Count  Months_Inactive_12_mon  \
count              10127.000000            10127.000000   
mean                   3.812580                2.341167   
std                    1.554408                1.010622   
min                    1.000000                0.000000   
25%                    3.000000                2.000000   
50%

In [26]:
# Droping CLIENTNUM columns because Client number is unique identifier of account holder. This feature doesn't really effect the attrition  prediction.
df.drop(['CLIENTNUM'], inplace=True, axis=1)

In [27]:
df.shape

(10127, 20)